# Multi-Agency Water Quality Data from the Water Quality Portal (WQP)

`dataretrieval` also allows users to access data from the [Water Quality Portal](http://www.waterqualitydata.us/). The WQP houses data from multiple agencies; while USGS data comes from the NWIS database, EPA data comes from the STORET database (this includes many state, tribal, NGO, and academic groups). The WQP brings data from all these organizations together and provides it in a single format that has a more verbose output than NWIS. To get non-NWIS data, need to use CharacteristicName instead of parameter code.


## WQP Basic Retrievals

Much like the convenience functions for NWIS, there's a simple function for retrievals if the site number and parameter code or characteristic name is known.

Both `dataretrieva.nwis` and `dataretrieva.wqp` allow users to pass arguments directly to the underlying REST API's; however `wqp` tends to be more bare bones.



In [ ]:
import pandas as pd
from dataretrieval import wqp

[i  for i in dir(wqp) if 'get' in i or 'what' in i]

In [ ]:
wqp.get_results?

## Large queries
Now returning to the problem from the previous notebook,
how might we construct a statewide query for phosphorus data?

WQP, like NWIS, has it's own idiosyncrasies.
In part because WQP's API is changing, `dataretrieval` makes less effort to hide these than for NWIS. Nevertheless, the API is very powerful if you have the doc close at hand.

In [ ]:
from dataretrieval.codes import fips_codes

# format FIPS (state) code for WQP
statecode = f"US:{fips_codes['Illinois']}"
statecode

In [ ]:
# now query WQP by state code
df, meta = wqp.get_results(
    statecode=statecode,
    pCode="00665", # total phosphorus
    minresults="200",
    providers="NWIS", # STORET data don't have pcodes
)

In [ ]:
df.shape

In [ ]:
n_samples = df.shape[0]
n_sites = df['MonitoringLocationIdentifier'].unique().shape[0]

print(f"The query returned {n_samples} samples from {n_sites} monitoring sites.")

In [ ]:
df.columns

We'll dig into some of these details a bit more later.
For now, let's do a little exploration.
Let's try plotting the temporal extent of these data.

First, `groupby` monitoring location, then compute the earliest date at each location.

In [ ]:
groupby = df.groupby('MonitoringLocationIdentifier')

start_dates = groupby['ActivityStartDate'].apply(
        lambda x: x.min()
)

start_dates = pd.to_datetime(start_dates)
start_dates.name = 'start'

and the end dates

In [ ]:
end_dates = groupby['ActivityStartDate'].apply(
        lambda x: x.max()
)

end_dates = pd.to_datetime(end_dates)
end_dates.name = 'end'

and now the differences.

In [ ]:
dates = pd.concat([start_dates, end_dates], axis=1)
dates['diff'] = dates['end'] - dates['start']
dates.head()

Every analysis should end in a visualization,
so here is a simple example of how we could visualize temporal data coverage in our region of interest.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

fig, ax = plt.subplots(figsize=(8,6))
 
y_tick_labels = dates.index.values
y_pos = np.arange(len(y_tick_labels))
 
ax.set_yticks(y_pos)
ax.set_yticklabels(y_tick_labels)
 
for index, row in dates.sort_values(by='start').reset_index().iterrows():
    start_year = int(row.start.strftime("%Y"))
    duration = row['diff'].days/365
    ax.broken_barh([(start_year, duration)], 
                    (index-0.5,0.8), 
                    facecolors =('tan'),
                   label=row.index)


## STORET data
In the last example, we queried data from NWIS using a parameter code or "p code",
which are eventually going away, so we'll need to query data by other means.
"P codes" can seem mysterious.
You might think, how the heck can I remember that "00665" is total phosphorus,
not to mention the other 25,000 codes?
That's a fair criticism, but as we'll see the alternatives bring their own challenges.

Before we dive in, consider these p codes:
00665 is total phosphorus in mg/L as P

In [ ]:
from dataretrieval import nwis
df, _ = nwis.get_pmcodes("00665")
df[['parameter_cd','parm_nm','parm_unit', 'SRSName']]

00631 is dissolved nitrate as mg/L as N.

In [ ]:
df, _ = nwis.get_pmcodes("00631")
df[['parameter_cd','parm_nm','parm_unit', 'SRSName']]

Each of these examples also shows the The Substance Registry Services (SRS) name,
which is an authoritative name for substances tracked or regulated by EPA (and used by WQP).

A p code doesn't doesn't tell us everything about a sample, but it tells us a lot:
- the substance (nitrate plus nitrite)
- what fraction (filtered, i.e, dissolved)
- the units (mg/l as N)

In other words, when we query NWIS with p codes, we filter our samples on each of these properties.
However, when we go to WQP, we are currently limited to the SNS name, which leaves some additional work for the user. 

Best to demonstrate by example. Let's query STORET for data at a particular site, 
then query a co-located USGS site and note some differences
(depending on class size, we can do this in parallel - permutations of USGS or IEPA, N or P).


Pull up the doc if you need a refresh;
otherwise, begin by all the characteristics at your site
(note the first difference: "parameters" versus "characteristics")

Watch out for some more typical WQP mistakes...

In [ ]:
df, _ = wqp.get_results(
    siteid="IL_EPA_WQX-D-32",
    #siteid="05586100"
)

List all available parameters/characteristics

In [ ]:
parameter_list = df["CharacteristicName"].unique().tolist()
parameter_list

Let's narrow our search to specific characteristics and periods.

In [ ]:
siteid="IL_EPA_WQX-D-32" # Illinois EPA monitoring site on the Illinois River
#siteid="USGS-05586100" # co-located USGS site

characteristic = 'Phosphorus'
#characteristic = 'Nitrate + Nitrite'
#characteristic = 'Inorganic nitrogen (nitrate and nitrite)'

df, _ = wqp.get_results(
    siteid=siteid,
    characteristicName=characteristic, # Note that we can't query by fraction
    startDateLo="1980-01-01",
)

Uh oh! Take a minute to debug what happened.

Reread the doc on `wqp.get_results` or else try to recreate this query at [waterqualitydata.us](https://www.waterqualitydata.us/#advanced=true).

Once you've found the mistake -- the date field was NWIS format -- rerun the query and continue on.

This one was easy, but with WQP it's much easier to create erroneous queries with multiple mistakes, which can be tricky to debug.

### WARNING 
WQP is a complex and changing service. Some of this pain should get better with time,
but in the meantime, it's still a great resouce; 
just anticipate that your codes may break periodically.
For this reason, `dataretrieval.wqp` offers fewer conveniences and instead gives a shallow wrapper around the webservice API.

Back to your query:

In [ ]:
print(f"Your query returned {df.shape[0]} samples!")

In [ ]:
df["ResultMeasure/MeasureUnitCode"].unique()

In [ ]:
df["ResultSampleFractionText"].unique()

Running a through a few iterations of different characteristics and databases,
some of the inconsistencies will quickly become apparanent. 
Each database might use a different nameing, unit, date-time convention, etc.
Use the wrong filter for your data, and your query might accidently return no data or, worse, the wrong data.
Be sure to watch out for this in your applications!